# Emissions 02: Full Pipeline Testing

**Purpose**: Test complete Emissions pipeline (Bronze → Silver → Gold)

**Outputs**:
- primes_emises_{vision}_pol_garp (by guarantee)
- primes_emises_{vision}_pol (aggregated by policy)

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("Emissions_Pipeline_Testing") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Initialize Processor

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.logger import PipelineLogger

config = ConfigLoader(str(project_root / "config" / "config.yml"))
logger = PipelineLogger("emissions_test")

VISION = "202509"
print(f"Testing pipeline for vision: {VISION}")

## 2. Run Emissions Processor

In [ ]:
from src.processors.emissions_processors import EmissionsProcessor

try:
    emissions_processor = EmissionsProcessor(spark, config, logger)
    result = emissions_processor.run(VISION)
    
    if result is not None:
        df_pol_garp, df_pol = result
        print(f"✓ POL_GARP (by guarantee): {df_pol_garp.count():,} rows")
        print(f"✓ POL (aggregated): {df_pol.count():,} rows")
    else:
        print("⚠ Processor returned None")
        df_pol_garp = df_pol = None
except Exception as e:
    print(f"✗ Error: {e}")
    df_pol_garp = df_pol = None

## 3. Verify Outputs

In [ ]:
if df_pol_garp is not None:
    print("POL_GARP output (by guarantee):")
    df_pol_garp.select('nopol', 'cdprod', 'cdgarp', 'prime_emise').show(5)
    
    print("\nPOL output (aggregated by policy):")
    df_pol.select('nopol', 'cdprod', 'prime_totale').show(5)
else:
    print("⚠ No data to display")

## Summary

In [ ]:
print("="*60)
print("EMISSIONS PIPELINE TESTING COMPLETE")
print("="*60)
print(f"\nVision: {VISION}")
print(f"POL_GARP: {'✓' if df_pol_garp is not None else '✗'}")
print(f"POL:      {'✓' if df_pol is not None else '✗'}")
print("\n→ Run: python main.py --vision 202509 --component emissions")